In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Reshape, Conv1D, MaxPooling1D, Dropout, Flatten, Lambda
from tensorflow.keras.optimizers import Adam  
from sklearn.model_selection import train_test_split



In [2]:
def load_data_from_folders(base_folder):
    data = []
    labels = []
    ground_truths = [0, 30, 60, 90, 120, 150, 180, 210, 240, 270, 300, 330]
    
    for i, ground_truth in enumerate(ground_truths):
        folder_path = os.path.join(base_folder, f'class_{i}')
        for filename in os.listdir(folder_path):
            if filename.endswith('.txt'):
                file_path = os.path.join(folder_path, filename)
                with open(file_path, 'r') as file:
                    
                    numbers = [float(num) for line in file for num in line.split(',')]
                    if len(numbers) == 507:  
                        data.append(numbers)
                        labels.append(ground_truth)
    return np.array(data), np.array(labels)



In [3]:
base_folder = 'B:\jupyter_notebook\localization'
data, labels = load_data_from_folders(base_folder)


In [5]:
index_of_30 = np.where(labels == 30)[0]  
if len(index_of_30) > 0:
    print(f"30度的第一个样本数据: {data[index_of_30[0]]}")
else:
    print("没有找到30度的样本")

30度的第一个样本数据: [ 2.990e+01  5.990e+00 -3.000e-02 -2.260e+00 -1.650e+00  2.390e+00
  1.180e+00 -5.200e-01 -1.100e+00  1.020e+00  1.550e+00 -3.300e-01
 -1.190e+00  3.202e+01  7.990e+00 -3.900e-01 -3.130e+00 -1.480e+00
  1.850e+00  1.250e+00 -9.700e-01 -1.350e+00  1.070e+00  9.100e-01
 -3.700e-01 -8.600e-01  3.044e+01  6.590e+00 -5.400e-01 -3.810e+00
  5.000e-02  2.440e+00  1.650e+00 -1.190e+00 -1.070e+00  1.250e+00
  1.600e+00 -5.500e-01 -1.680e+00  3.420e+01  8.390e+00 -2.900e-01
 -2.810e+00 -0.000e+00  2.450e+00  1.040e+00 -1.540e+00 -1.230e+00
  7.900e-01  1.420e+00 -5.000e-02 -1.410e+00  3.309e+01  7.960e+00
 -3.100e-01 -2.750e+00  3.000e-02  1.790e+00  1.170e+00 -1.340e+00
 -8.200e-01  7.000e-01  9.500e-01 -7.800e-01 -1.630e+00  3.285e+01
  7.490e+00  5.800e-01 -2.450e+00 -1.400e-01  3.550e+00  1.450e+00
 -1.140e+00 -1.860e+00  7.500e-01  1.550e+00  4.000e-02 -1.140e+00
  3.351e+01  7.110e+00 -6.400e-01 -3.110e+00 -1.360e+00  2.000e+00
  9.600e-01 -2.700e-01 -1.220e+00 -2.300e-01  5.1

In [6]:

train_data, val_data, train_labels, val_labels = train_test_split(data, labels, test_size=0.2, random_state=42)

In [7]:

BATCH_SIZE = 32
train_dataset = tf.data.Dataset.from_tensor_slices((train_data, train_labels)).shuffle(buffer_size=len(train_data)).batch(BATCH_SIZE)
validation_dataset = tf.data.Dataset.from_tensor_slices((val_data, val_labels)).batch(BATCH_SIZE)


input_length = 507  

In [8]:
def cyclic_mse(y_true, y_pred):
    pi = tf.constant(np.pi, dtype=tf.float32)
    deg = tf.constant(180.0, dtype=tf.float32)  
    y_true = tf.cast(y_true, dtype=tf.float32)  
    y_pred = tf.cast(y_pred, dtype=tf.float32)  #y_pred to float32
    return tf.reduce_mean(tf.square(
        tf.atan2(
            tf.sin(y_true * pi / deg - y_pred * pi / deg),
            tf.cos(y_true * pi / deg - y_pred * pi / deg)
        )
    ) * deg / pi)


In [9]:
model = Sequential()
model.add(Reshape((int(input_length / 13), 13), input_shape=(input_length, )))
model.add(Conv1D(8, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2, strides=2, padding='same'))
model.add(Dropout(0.25))
model.add(Conv1D(16, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2, strides=2, padding='same'))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(1))  
model.add(tf.keras.layers.Lambda(lambda x: x % 360))  

D:\apps\anaconda\anaconda\envs\812_tf_env\lib\site-packages\keras\src\layers\reshaping\reshape.py:39: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [10]:
LEARNING_RATE = 0.005
opt = Adam(learning_rate=LEARNING_RATE, beta_1=0.9, beta_2=0.999)
model.compile(loss=cyclic_mse, optimizer=opt, metrics=[cyclic_mse])

In [11]:

EPOCHS = 300
model.fit(train_dataset, epochs=EPOCHS, validation_data=validation_dataset, verbose=2)
# the following printed results are just for testing

Epoch 1/300
18/18 - 1s - 70ms/step - cyclic_mse: 192.3902 - loss: 189.5540 - val_cyclic_mse: 176.6198 - val_loss: 172.8100
Epoch 2/300
18/18 - 0s - 3ms/step - cyclic_mse: 189.6441 - loss: 176.0926 - val_cyclic_mse: 178.0811 - val_loss: 159.1979
Epoch 3/300
18/18 - 0s - 3ms/step - cyclic_mse: 190.2017 - loss: 152.4727 - val_cyclic_mse: 174.6920 - val_loss: 123.5983
Epoch 4/300
18/18 - 0s - 3ms/step - cyclic_mse: 188.4356 - loss: 127.2279 - val_cyclic_mse: 178.9519 - val_loss: 104.2814
Epoch 5/300
18/18 - 0s - 3ms/step - cyclic_mse: 185.5848 - loss: 110.3483 - val_cyclic_mse: 176.4474 - val_loss: 93.6475
Epoch 6/300
18/18 - 0s - 3ms/step - cyclic_mse: 187.1445 - loss: 117.7756 - val_cyclic_mse: 186.6561 - val_loss: 128.7090
Epoch 7/300
18/18 - 0s - 4ms/step - cyclic_mse: 187.4951 - loss: 110.8799 - val_cyclic_mse: 173.7781 - val_loss: 95.1797
Epoch 8/300
18/18 - 0s - 3ms/step - cyclic_mse: 186.9240 - loss: 106.8881 - val_cyclic_mse: 178.9721 - val_loss: 97.5285
Epoch 9/300
18/18 - 0s - 4

In [12]:

predictions = model.predict(val_data)


5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step


In [15]:
def cyclic_absolute_difference(y_true, y_pred):
    pi = np.pi
    deg = 180.0
    
    
    y_true_radians = y_true * pi / deg
    y_pred_radians = y_pred * pi / deg
    
   
    angle_diff = np.arctan2(
        np.sin(y_true_radians - y_pred_radians),
        np.cos(y_true_radians - y_pred_radians)
    )
    
   
    angle_diff_deg = np.abs(angle_diff * deg / pi)
    
    return angle_diff_deg


In [18]:

cyclic_abs_diff_list = []

# 
for i in range(len(predictions)):
    y_pred = predictions[i][0]  
    y_true = val_labels[i]     

    
    if y_true != 1:
        abs_diff_value = cyclic_absolute_difference(y_true, y_pred)  
        cyclic_abs_diff_list.append(abs_diff_value)
        print(f"Prediction: {y_pred:.2f}, Ground Truth: {y_true}, Absolute Angle Difference: {abs_diff_value:.2f} degrees")


average_abs_diff = np.mean(cyclic_abs_diff_list)
print(f"Average Absolute Angle Difference: {average_abs_diff:.2f} degrees")



Prediction: 23.01, Ground Truth: 30, Absolute Angle Difference: 6.99 degrees
Prediction: 129.37, Ground Truth: 150, Absolute Angle Difference: 20.63 degrees
Prediction: 359.67, Ground Truth: 0, Absolute Angle Difference: 0.33 degrees
Prediction: 328.17, Ground Truth: 300, Absolute Angle Difference: 28.17 degrees
Prediction: 239.85, Ground Truth: 240, Absolute Angle Difference: 0.15 degrees
Prediction: 279.76, Ground Truth: 240, Absolute Angle Difference: 39.76 degrees
Prediction: 41.02, Ground Truth: 30, Absolute Angle Difference: 11.02 degrees
Prediction: 311.14, Ground Truth: 240, Absolute Angle Difference: 71.14 degrees
Prediction: 32.23, Ground Truth: 30, Absolute Angle Difference: 2.23 degrees
Prediction: 329.42, Ground Truth: 330, Absolute Angle Difference: 0.58 degrees
Prediction: 291.29, Ground Truth: 240, Absolute Angle Difference: 51.29 degrees
Prediction: 15.88, Ground Truth: 30, Absolute Angle Difference: 14.12 degrees
Prediction: 346.75, Ground Truth: 330, Absolute Angle D

In [19]:

model.save('B:/jupyter_notebook/localization/local_model.h5')


In [20]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ reshape (Reshape)                    │ (None, 39, 13)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d (Conv1D)                      │ (None, 39, 8)               │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d (MaxPooling1D)         │ (None, 20, 8)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 20, 8)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_1 (Conv1D)                    │ (None, 20, 16)              │             400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_1 (MaxPooling1D)       │ (None, 10, 16)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 10, 16)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 160)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │             161 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lambda (Lambda)                      │ (None, 1)                   │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,645 (10.34 KB)

 Trainable params: 881 (3.44 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,764 (6.89 KB)